In [19]:
import tensorflow as tf

In [20]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet import ResNet50

from keras import layers

In [3]:
import numpy as np

In [21]:
(train_examples, validation_examples, test_examples), info = tfds.load(
    'colorectal_histology', 
    with_info=True, 
    as_supervised=True, 
    split=['train[:80%]', 'train[80%:95%]', 'train[95%:]'],
)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [5]:
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [15]:
train_batches

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [16]:
validation_batches

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [1]:
ResNet50_model = ResNet50(weights='imagenet')

NameError: name 'ResNet50' is not defined

In [11]:
ResNet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [6]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/5", 
                                            output_shape=[2048], 
                                            trainable=False), 
                             layers.Dense(num_classes, activation='softmax')
])

model.build([None, IMAGE_RES, IMAGE_RES, 3])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23561152  
                                                                 
 dense (Dense)               (None, 8)                 16392     
                                                                 
Total params: 23,577,544
Trainable params: 16,392
Non-trainable params: 23,561,152
_________________________________________________________________


In [22]:
ResNet50_model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

ResNet50_model.fit(train_batches, epochs=10, validation_data=validation_batches)

Epoch 1/10
125/125 [==============================] - 50s 338ms/step - loss: 0.8756 - accuracy: 0.7715 - val_loss: 7.3394 - val_accuracy: 0.1320
Epoch 2/10
125/125 [==============================] - 42s 333ms/step - loss: 0.3756 - accuracy: 0.8708 - val_loss: 4.3330 - val_accuracy: 0.1307
Epoch 3/10
125/125 [==============================] - 42s 338ms/step - loss: 0.2674 - accuracy: 0.9060 - val_loss: 6.5935 - val_accuracy: 0.1240
Epoch 4/10
125/125 [==============================] - 42s 339ms/step - loss: 0.2042 - accuracy: 0.9298 - val_loss: 4.0386 - val_accuracy: 0.2040
Epoch 5/10
125/125 [==============================] - 42s 338ms/step - loss: 0.1712 - accuracy: 0.9408 - val_loss: 4.3505 - val_accuracy: 0.1707
Epoch 6/10
125/125 [==============================] - 42s 338ms/step - loss: 0.1578 - accuracy: 0.9460 - val_loss: 5.1355 - val_accuracy: 0.1560
Epoch 7/10
125/125 [==============================] - 43s 340ms/step - loss: 0.1099 - accuracy: 0.9632 - val_loss: 7.6636 - val_ac

In [23]:
score = ResNet50_model.evaluate(train_batches, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7733266353607178
Test accuracy: 0.8327500224113464


In [24]:
ResNet50_model.evaluate(test_batches)

8/8 [==============================] - 2s 183ms/step - loss: 0.9120 - accuracy: 0.8240


[0.9120102524757385, 0.8240000009536743]